In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\shaha\Desktop\IBS\train.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords = nltk.corpus.stopwords.words('english')
import string
stemmer = SnowballStemmer("english")
df['new_tweet'] = df.tweet.str.replace('@user', '')
df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )
df['new_tweet'] = df['new_tweet'].str.replace("#", "")
df['new_tweet'] = df['new_tweet'].apply(lambda x: x.split())
df['new_tweet']= df['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
df['new_tweet'] = df['new_tweet'].apply(process)
df.head()


<ipython-input-2-616026ddc9b1>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )


,id,label,tweet,new_tweet
0,1,0,@user when a father is dysfunctional and is s...,"[selfish, father, drag, run, kid, dysfunct]"
1,2,0,@user @user thanks for #lyft credit i can't us...,"[thank, credit, caus, disapoint, lyft, use, of..."
2,3,0,bihday your majesty,"[bihday, majesti]"
3,4,0,#model i love u take with u all the time in ...,"[time, love, model, ur, u, take]"
4,5,0,factsguide: society now #motivation,"[factsguid, societi, motiv]"


In [3]:
df=df.drop(columns='tweet')

In [4]:
df=df.drop(columns='id')


In [5]:
import numpy as np
N = 10    
split=np.array_split(df, N)

In [6]:
print(split[5])

       label                                          new_tweet
15982      0                               [thank, rain, posit]
15983      0  [anxieti, power, free, podcast, nowplay, stres...
15984      0         [whit, pictur, photographi, princess, tbt]
15985      1  [accept, ref, hispanicvot, know, blackvot, pro...
15986      0     [got, cawheel, damien, saw, dandi, today, hug]
...      ...                                                ...
19173      0  [view, bieszczadymountain, frogg, everywher, d...
19174      0  [ed, rose, ty, love, ved, chronicpain, much, f...
19175      0                           [home, happi, big, hous]
19176      0  [nuke, hall, soon, amp, limit, world, everi, r...
19177      0  [onli, instalik, day, smile, instafollow, good...

[3196 rows x 2 columns]


In [7]:
split[1]['new_tweet']

3197    [make, scifi, read, surpris, ka, betray, lt, g...
3198    [least, final, back, much, mystiqu, warriorsfa...
3199    [make, imperfect, learn, forgiv, quot, inspir,...
3200    [direct, bull, whatev, want, lightroom, domin,...
3201    [truli, summer, poster, day, stas, grahamepark...
                              ...                        
6389               [eat, waiter, want, aswel, plate, ask]
6390    [view, moment, mamashelt, turn, life, smile, n...
6391    [doubl, cute, tag, petsagram, hound, best, dog...
6392    [antiegalitarian, climatechang, fascism, narci...
6393    [weekend, time, forget, dear, nice, spoy, take...
Name: new_tweet, Length: 3197, dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
def tfidf(x):
    count_vect = CountVectorizer(stop_words='english')
    transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
    x_counts = count_vect.fit_transform(x.apply(lambda x: ' '.join(x)))
    x_tfidf = transformer.fit_transform(x_counts)
    return x_tfidf

In [9]:

from modAL.batch import uncertainty_batch_sampling
from functools import partial
from modAL.models import ActiveLearner
from sklearn.linear_model import LogisticRegression
from modAL.uncertainty import entropy_sampling
BATCH_SIZE = 100
preset_batch = partial(uncertainty_batch_sampling, n_instances=BATCH_SIZE)

In [10]:
learner = ActiveLearner(estimator=LogisticRegression(),query_strategy=entropy_sampling,X_training=tfidf(split[1]['new_tweet']), y_training=split[1]['label'])

In [11]:
tfidf(split[8]['new_tweet'])

<3196x6972 sparse matrix of type '<class 'numpy.float64'>'
	with 21598 stored elements in Compressed Sparse Row format>

In [12]:
for i in range(2,10):
  learner.fit(tfidf(split[i]['new_tweet']),split[i]['label'])
  predictions = learner.predict(tfidf(split[i]['new_tweet']))
  accuracy_scores = [learner.score(tfidf(split[i]['new_tweet']), split[i]['label'])]
  print(accuracy_scores)

[0.9355444305381727]
[0.9361702127659575]
[0.9342928660826032]
[0.9399249061326659]
[0.925531914893617]
[0.931476846057572]
[0.9374217772215269]
[0.9352315394242804]
